## Read Original Dataset mt bench, convert to datasets

In [8]:
import json
import datasets

mt_bench_en = "./datasets/final_evaluation_datasets/mt_bench.jsonl"

mt_bench_list = []
with open(mt_bench_en) as f:
    mt_bench_list = [json.loads(line) for line in f.readlines()]

mt_bench_en = datasets.Dataset.from_list(mt_bench_list)
mt_bench_en[0]

{'question_id': 81,
 'category': 'writing',
 'turns': ['Compose an engaging travel blog post about a recent trip to Hawaii, highlighting cultural experiences and must-see attractions.',
  'Rewrite your previous response. Start every sentence with the letter A.']}

In [ ]:
mt_bench_en.push_to_hub("dim/mt_bench_en")

## Evaluate original saiga

In [3]:
from datasets import load_dataset
from tqdm import tqdm
import time
import shortuuid
import json

mt_bench_en = load_dataset("dim/mt_bench_en")
mt_bench_en = mt_bench_en["train"]
mt_bench_en = mt_bench_en.to_list()
mt_bench_en[0]

Found cached dataset parquet (/home/kosenko/.cache/huggingface/datasets/dim___parquet/dim--mt_bench_en-40a2837a0335d6fc/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
100%|██████████| 1/1 [00:00<00:00, 636.08it/s]


{'question_id': 81,
 'category': 'writing',
 'turns': ['Compose an engaging travel blog post about a recent trip to Hawaii, highlighting cultural experiences and must-see attractions.',
  'Rewrite your previous response. Start every sentence with the letter A.']}

In [4]:

def convert_item_mt_bench_answer(
    item=None,
    model_id=None,
):
    assert model_id
    return {
        "question_id": item["question_id"],
        "answer_id": shortuuid.uuid(),
        "model_id": model_id,
        "choices": [
            {
                "index": 0,
                "turns": item["replies"],
            }
        ],
        "tstamp": time.time(),
    }


def convert_mt_bench_format(
    old_filename,
    model_id,
):
    with open(old_filename) as f:
        gen_dataset = json.load(f)

    new_dataset = []
    for item in gen_dataset:
        item = convert_item_mt_bench_answer(
            item=item,
            model_id=model_id,
        )
        new_dataset.append(item)

    converted_path = old_filename.split(".json")[0]
    converted_path = f"{converted_path}_converted.jsonl"
    # print(converted_path)
    with open(converted_path, "w") as f:
        for item in new_dataset:
            json.dump(item, f)
            f.write("\n")
        # f.write("\n")


convert_mt_bench_format(
    # "./datasets/final_evaluation_datasets/mt_bench/mt_bench_en_saiga_7b.json",
    # "./datasets/final_evaluation_datasets/mt_bench/mt_bench_en_saiga_7b_v2.json",
    "datasets/final_evaluation_datasets/mt_bench/mt_bench_en_xglm_4.5b_v10_epoch_6_step_41141.json",
    # model_id="saiga_7b_v1",
    # model_id="saiga_7b_v2",
    model_id="xglm_4.5b_v10_epoch_6_step_41141",
)

In [ ]:
# mt bench format original
{
    "question_id": 81,
    "answer_id": "VtuVQhCuDYtdyyNPaQKfF4",
    "model_id": "vicuna-7b-v1.3",
    "choices": [
        {
            "index": 0,
            "turns": [
                "Aloha! I recently returned from a trip to Hawaii and it was an absolute dream come true. From the stunning natural beauty to the rich cultural experiences, there was so much to see and do on this tropical paradise.\n\nOne of the standout attractions was the Hawaii Volcanoes National Park. The park is home to two of the world's most active volcanoes, Kilauea and Mauna Loa, and offers a unique opportunity to witness the power of nature up close. We took a guided hike to the summit of Kilauea, where we were able to see the lava flowing from the crater and feel the heat of the earth. It was an awe-inspiring experience that I will never forget.\n\nAnother highlight of our trip was learning about the Hawaiian culture at the Bishop Museum. The museum is dedicated to preserving the history and traditions of Hawaii and offers a glimpse into the island's rich cultural heritage. We learned about the history of the Hawaiian monarchy, the traditional art of hula, and the significance of Hawaiian religion and spirituality.\n\nIn terms of food, we were blown away by the diverse and delicious flavors of Hawaiian cuisine. From poke bowls to shrimp shacks to traditional Hawaiian dishes like kalua pig and laulau, there was something for everyone. We also had the opportunity to try some local brews, including the famous Kona Brewing Co. beers.\n\nOverall, our trip to Hawaii was an unforgettable experience that we will always treasure. From the stunning natural beauty to the rich cultural experiences, there was something for everyone to enjoy. I highly recommend adding Hawaii to your travel bucket list!",
                "Aloha! Recently, I visited Hawaii and it was an absolutely amazing experience. The natural beauty of the islands is stunning, and there is so much to see and do. A must-see attraction was the Hawaii Volcanoes National Park, where we were able to witness the power of nature up close. We took a guided hike to the summit of Kilauea, where we could see the lava flowing from the crater and feel the heat of the earth. Another highlight of our trip was learning about the Hawaiian culture at the Bishop Museum. The museum offers a glimpse into the island's rich cultural heritage, including the history of the Hawaiian monarchy, traditional art of hula, and the significance of Hawaiian religion and spirituality.\n\nIn terms of food, we were blown away by the diverse and delicious flavors of Hawaiian cuisine. From poke bowls to shrimp shacks to traditional Hawaiian dishes like kalua pig and laulau, there was something for everyone. We also had the opportunity to try some local brews, including the famous Kona Brewing Co. beers.\n\nOverall, our trip to Hawaii was an unforgettable experience that we will always treasure. From the stunning natural beauty to the rich cultural experiences, there was something for everyone to enjoy. I highly recommend adding Hawaii to your travel bucket list!",
            ],
        }
    ],
    "tstamp": 1689675846.1825185,
}

In [ ]:
# my generated content
{
    "question_id": 81,
    "category": "writing",
    "turns": [
        "Compose an engaging travel blog post about a recent trip to Hawaii, highlighting cultural experiences and must-see attractions.",
        "Rewrite your previous response. Start every sentence with the letter A.",
    ],
    "replies": [
        'Hawaii: A Tropical Paradise of Culture and Adventure\n\nI recently returned from my dream vacation in the tropical paradise of Hawaii. This beautiful island chain is known for its pristine beaches, lush rainforests, and rich culture. I spent two weeks exploring the islands of Oahu, Maui, Kauai, and Big Island, and had the time of my life! Here are some of my favorite cultural experiences and must-see attractions that I recommend you add to your itinerary if you ever visit this magical place.\n\nOahu: The Heart of Hawaiian Culture\n\nThe first stop on my journey was Oahu, the most populous island in the chain. It\'s also home to Honolulu, the capital city of Hawaii. One of the best ways to experience the local culture is by taking a tour of the historic Pearl Harbor Memorial. You can learn all about the attack on Pearl Harbor during World War II and see the USS Arizona Memorial, which sits atop the sunken battleship. Another great way to get a taste of Hawaiian history is by visiting the Polynesian Cultural Center, where you can watch traditional dances and performances and learn about the different cultures that make up Hawaii today.\n\nMaui: The Valley Isle\n\nNext, I headed over to Maui, also known as "the Valley Isle" because of its fertile central valley. One of the top things to do here is hike to Haleakala National Park, which features breathtaking views of the Pacific Ocean and the volcanic crater. If you\'re feeling adventurous, you can even sign up for a sunrise hike to catch the spectacular view of the rising sun. Another must-visit attraction is the Road to Hana, a scenic drive along the coastline with plenty of stops to take photos and explore waterfalls and beaches.\n\nKauai: The Garden Isle\n\nAfter leaving Maui, I traveled to Kauai, also known as "the Garden Isle." This island is famous for its lush greenery and stunning mountain ranges. One of the best ways to experience the beauty of Kauai is by taking a helicopter tour. You can fly over the Na Pali Coast, one of the most picturesque areas in the world, and see the dramatic cliffs and waterfalls up close. Another must-do activity is hiking to Waimea Canyon State Park, which features incredible views of the canyon and surrounding mountains.\n\nBig Island: Volcanoes and Black Sand Beaches\n\nFinally, I ended my trip on the Big Island, which is home to active volcanoes and black sand beaches. One of the best ways to experience the volcanoes is by taking a tour of the Hawaii Volcanoes National Park. You can see lava flows and learn about the geological history of the island. Another must-visit attraction is the Mauna Kea Observatory, which is home to some of the largest telescopes in the world. To end your day, you should definitely check out the black sand beach at Punaluu, which is famous for its unique landscape and wildlife.\n\nIn conclusion, Hawaii is truly a magical destination full of culture and adventure. Whether you want to learn about the history of the islands or just relax on the beach, there is something for everyone. I highly recommend adding these attractions to your itinerary if you ever decide to visit this amazing place. Aloha!',
        "A is for adventure, and I am ready for it! A is for airplanes, and I love flying. A is for activities, and I will try them all. A is for adrenaline, and I need more of it. A is for amazing views, and I cannot wait to see them. A is for authentic food, and I will eat it all. A is for awesome people, and I hope to meet them. A is for adventures, and I am ready for them!",
    ],
}